# 01 — SEEK Perth Jobs: Data Cleaning & EDA

This notebook loads SEEK job-card data (scraped) and produces a cleaned dataset suitable for analysis.

**Outputs**
- `data/cleaned/seek_cleaned_perth_jobs.xlsx`

**Notes**
- SEEK job cards often mix location, posted time (e.g., `6d ago`), and `Featured` flags. We separate these.


In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt

# Use relative paths so this works on your machine after cloning
RAW_PATH = '../data/raw/seek.xlsx'
CLEAN_PATH = '../data/cleaned/seek_cleaned_perth_jobs.xlsx'

raw = pd.read_excel(RAW_PATH)
raw.head()

## 1) Quick inspection
The scraped sheet usually has anonymous column names because they come from web DOM structure.
We rename them to `raw_0..raw_n` for consistent processing.

In [ ]:
raw.columns = [f'raw_{i}' for i in range(len(raw.columns))]
raw.shape, raw.columns.tolist()

## 2) Column mapping (job-card level)
Based on observed patterns:
- `raw_0`: Job title
- `raw_2`: Company
- `raw_6`: Category (e.g., `(Sales)`)
- `raw_3`/`raw_4`: Employment type text (often `This is a Full time job`)
- `raw_5`/`raw_8`: Location text (Perth/WA/Suburb)
- `raw_7`: Posted time (e.g., `6d ago`) or `Featured`
- `raw_9`: Salary (if present)


In [ ]:
clean = pd.DataFrame()
clean['job_title'] = raw['raw_0']
clean['company'] = raw['raw_2']
clean['category'] = raw['raw_6']

# Employment type: prefer raw_3 if it contains Full/Part, else fallback to raw_4
clean['employment_type'] = raw['raw_3'].where(raw['raw_3'].str.contains('Full|Part', na=False), raw['raw_4'])
clean['employment_type'] = clean['employment_type'].astype('string').str.replace('This is a ', '', regex=False)

# Location: prefer raw_5 if it looks like a location, else fallback to raw_8
clean['location'] = raw['raw_5'].where(raw['raw_5'].str.contains('WA|Perth', na=False), raw.get('raw_8'))

def extract_days(x):
    if isinstance(x, str):
        m = re.search(r'(\d+)\s*d ago', x)
        if m:
            return int(m.group(1))
    return pd.NA

clean['posted_days'] = raw['raw_7'].apply(extract_days).astype('Int64')
clean['featured'] = raw['raw_7'].eq('Featured')
clean['salary'] = raw.get('raw_9')

# Remove empty titles and duplicates
clean = clean[clean['job_title'].notna()].drop_duplicates()
clean.shape

## 3) Data quality checks
We check:
- Missing values
- Reasonable ranges for `posted_days`
- Top titles/companies/locations


In [ ]:
clean.isna().mean().sort_values(ascending=False)

In [ ]:
clean['posted_days'].describe()

In [ ]:
clean['job_title'].value_counts().head(15)

In [ ]:
clean['company'].value_counts().head(15)

In [ ]:
clean['location'].value_counts().head(15)

## 4) Simple EDA charts
A few quick visual checks. (No special styling; keep it simple.)

In [ ]:
# Posted-days distribution (ignoring missing)
tmp = clean['posted_days'].dropna()
plt.figure()
plt.hist(tmp, bins=30)
plt.title('Distribution of posted_days')
plt.xlabel('Days since posted')
plt.ylabel('Count')
plt.show()

In [ ]:
# Top 15 job titles
top_titles = clean['job_title'].value_counts().head(15)[::-1]
plt.figure(figsize=(8,5))
plt.barh(top_titles.index, top_titles.values)
plt.title('Top 15 Job Titles')
plt.xlabel('Count')
plt.ylabel('Job title')
plt.tight_layout()
plt.show()

In [ ]:
# Featured vs non-featured
feat_counts = clean['featured'].value_counts()
plt.figure()
plt.bar(['Non-featured', 'Featured'], [feat_counts.get(False,0), feat_counts.get(True,0)])
plt.title('Featured vs Non-featured')
plt.ylabel('Count')
plt.show()

## 5) Export cleaned dataset
Export to Excel so you can analyze in Excel/Power BI or commit it to the repo (if you choose).

> Tip: If you don't want to commit data files to Git, add `data/` to `.gitignore` and only keep notebooks + scripts.

In [ ]:
clean.to_excel(CLEAN_PATH, index=False)
print('Saved:', CLEAN_PATH)

## 6) Next steps
- Add keyword tagging (Sales/BD/Account/etc.)
- Normalize locations (Perth WA vs suburbs)
- Build a dashboard (Power BI/Tableau)
- Add job URL + job ID if you scrape detail pages
